In [121]:
import json
import math
import pandas as pd

What we're aiming for is something like this

```json
{
  "E14999999":{
    "name": "Fake news",
    "code": "E14999999",
    "description": "This constituency is great!",
    "headlines": [
      {"value":1,"name":"People"},
      {"value":1,"name":"People"},
      {"value":1,"name":"People"},
      {"value":1,"name":"People"}
    ],
		"similar": { "People": 4 },
		"leastsimilar": { "People": 1000 },
		"neet": {
			"something": [34,56,24,19,25],
			"another": {}
		},
		"employment": {
			
		}
	}
}
```

In [122]:
descriptions=pd.read_csv('../../data/reference/constituency-descriptions.csv', index_col='PCON22CD').fillna('')

In [123]:
pcons=pd.read_json('../../src/_data/areas/reference/pcon.json')

In [124]:
summary=pd.merge(pcons, descriptions, how='left', left_on='PCON22CD', right_index=True).set_index('PCON22CD')

summary.rename(columns={ 'PCON22NM': 'name'}, inplace=True)
summary.rename(columns=lambda x: x.lower(), inplace=True)

In [125]:
summary = summary.to_dict(orient='index')

In [126]:
employment_headlines = pd.read_csv('../../data/area/pcon/headlines.csv', index_col='PCON22CD')
education_headlines = pd.read_csv('../../data/area/pcon/education_attainment_pcon_2010.csv', index_col='PCON22CD')


econ_inactive = pd.read_csv('../../data/area/pcon/econ_inactive_16-24_last_3_years.csv', index_col='PCON22CD')
unemployment = pd.read_csv('../../data/area/pcon/youth_unem_16-24_last_3_years.csv', index_col='PCON22CD')
economic_activity_16_19 = pd.read_csv('../../data/area/pcon/econ_active_16-19_last_3_years.csv', index_col='PCON22CD')
economic_activity_20_24 = pd.read_csv('../../data/area/pcon/econ_active_20-24_last_3_years.csv', index_col='PCON22CD')

In [127]:
education_headlines

,Average Attainment 8 Score,Average Progress 8 Score,Percentage Achieving Grade 4 English & Maths,Percentage Achieving Grade 5 English & Maths,suffix
PCON22CD,,,,,
E14000530,40.9,-0.63,56.5,35.8,%
E14000531,42.3,-0.36,58.0,35.3,%
E14000532,59.5,0.40,82.9,67.4,%
E14000533,43.4,-0.15,60.0,38.4,%
E14000534,47.9,0.09,67.8,46.5,%
...,...,...,...,...,...
E14001058,41.8,-0.27,56.4,38.5,%
E14001059,44.4,-0.42,57.8,41.6,%
E14001060,43.0,-0.28,60.5,39.0,%


In [128]:
for s in summary:
    suffix = employment_headlines.loc[s, :].Suffix
    summary[s]['employment_headlines'] = [
        { 'h': headline_name, 'v': headline_value, 'suffix': suffix }
        for headline_name, headline_value
        in employment_headlines.loc[s, :].drop(labels="Suffix").items()
        ]
    summary[s]['education_headlines'] = [
        { 'h': name, 'v': value, 'suffix': suffix}
        for name, value
        in education_headlines.loc[s, :].drop(labels="suffix").items()
        ]
    try: 
        summary[s]['economic_inactivity'] = [
            { 'year': date_name, 'value': value }
            for date_name, value
            in econ_inactive.loc[s,:]
        ]
        # summary[s]['economic_activity_16_19'] = [
        #     { 'year': date_name, 'value': value }
        #     for date_name, value
        #     in economic_activity_16_19.loc[s,:].items()
        # ]
        # summary[s]['economic_activity_20_24'] = [
        #     { 'year': date_name, 'value': value }
        #     for date_name, value
        #     in economic_activity_20_24.loc[s,:].items()
        # ]
    except: 
        print(f'No data found for constituency: {s}')
    
    '''Do this for all other things...'''

No data found for constituency: E14000530
No data found for constituency: E14000531
No data found for constituency: E14000532
No data found for constituency: E14000533
No data found for constituency: E14000534
No data found for constituency: E14000535
No data found for constituency: E14000536
No data found for constituency: E14000537
No data found for constituency: E14000538
No data found for constituency: E14000539
No data found for constituency: E14000540
No data found for constituency: E14000541
No data found for constituency: E14000542
No data found for constituency: E14000543
No data found for constituency: E14000544
No data found for constituency: E14000545
No data found for constituency: E14000546
No data found for constituency: E14000547
No data found for constituency: E14000548
No data found for constituency: E14000549
No data found for constituency: E14000550
No data found for constituency: E14000551
No data found for constituency: E14000552
No data found for constituency: E1

In [129]:
with open('../../src/generated/areas/constituency/_data/summary.json', 'w') as f:
    json.dump(summary, f, indent=2)